# Credit Score

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

# Database Description

|Variables            |Descripción|
|---------------------|---|
|ID                   |Llave de identificación|
|Periodo              |Fecha de Observación (AAAAMM)|
|Dias 0-6             |Dias0: cantidad de días en mora, a la fecha de observación|
| |Dias1: cantidad de días en mora, un mes después de la observación.| 
| |...|
| |Dias6: cantidad de días en mora, seis meses después de la observación|
|Número_Producto      |Número de productos a la fecha|
|Genero               |Género (M-F)|
|Saldo_Mora 1-6(Pesos)|Saldo_Mora1: saldo del crédito en mora, un mes después de la observación|
|| ...|
|| Saldo_Mora6: saldo del crédito en mora, seis meses después de la observación|
|Cupo (Pesos)         |Monto del crédito otorgado|
|Saldo 1-6 (Pesos)    |Saldo1: saldo del crédito, un mes después de la observación|
|| ...|
|| Saldo6: saldo del crédito, seis meses después de la observación|

# 1. Data Pre-Processing

In [2]:
df = pd.read_csv('Anonimizada_DataBase.txt', delimiter = '|')

In [3]:
df.head()

,Identificador,periodo,DIAS1,DIAS2,DIAS3,DIAS4,DIAS5,DIAS6,DIAS0,Numero_Producto,...,Saldo1,Saldo_Mora3,Saldo2,Saldo_Mora4,Saldo3,Saldo_Mora5,Saldo4,Saldo_Mora6,Saldo5,Saldo6
0,92966525716,201709,7.0,6.0,0.0,0.0,0.0,4.0,5,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,92967390638,201709,0.0,0.0,0.0,0.0,0.0,0.0,0,1,...,"14493557,79","294570,52","14383360,76","294569,96","14271872,86","294569,05","14159078,99","294567,62","14044963,84","13929511,94"
2,92967489284,201712,0.0,0.0,0.0,0.0,0.0,0.0,0,1,...,NaN,0,0,12262,3524442,49553,"3516034,8",41122,"2989794,73","2830734,73"
3,92966459151,201809,0.0,0.0,0.0,0.0,0.0,NaN,0,2,...,9738321.38,32433,32433,32433,32433,32433,32433,32433,32433,0
4,92966529824,201712,0.0,0.0,0.0,0.0,0.0,5.0,4,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 24 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Identificador    30000 non-null  int64  
 1   periodo          30000 non-null  int64  
 2   DIAS1            29213 non-null  float64
 3   DIAS2            28616 non-null  float64
 4   DIAS3            28070 non-null  float64
 5   DIAS4            27369 non-null  float64
 6   DIAS5            26792 non-null  float64
 7   DIAS6            22642 non-null  float64
 8   DIAS0            30000 non-null  int64  
 9   Numero_Producto  30000 non-null  int64  
 10  GENERO           23819 non-null  object 
 11  Saldo_Mora1      21592 non-null  object 
 12  Cupo             21592 non-null  object 
 13  Saldo_Mora2      22213 non-null  object 
 14  Saldo1           22213 non-null  object 
 15  Saldo_Mora3      22877 non-null  object 
 16  Saldo2           22877 non-null  object 
 17  Saldo_Mora4 

## Se deben hacer cambios de tipo de datos en algunas columnas. La columna Periodo, debe tener formato de fecha y las columnas Cupo y Saldo y DIAS0 deben ser tipo float. 

### Periodo

In [5]:
df['periodo'].dtypes

dtype('int64')

In [6]:
df['periodo'] = pd.to_datetime(df['periodo'], format='%Y%m', errors='coerce').dt.to_period('m')

In [7]:
df['periodo'].dtypes

period[M]

### Cupo y Saldo

In [8]:
columns = ['Cupo', 'Saldo_Mora1', 'Saldo_Mora2', 'Saldo_Mora3', 'Saldo_Mora4', 
           'Saldo_Mora5', 'Saldo_Mora6', 'Saldo1', 'Saldo2', 'Saldo3', 'Saldo4', 
           'Saldo5', 'Saldo6']
for colum in columns:
    df[colum] = pd.to_numeric(df[colum], downcast='float', errors = 'coerce').astype(float)

### DIAS0

In [9]:
df['DIAS0'] = df['DIAS0'].astype(float)

## Index. Se define el index de la base de datos con la columna Identificador. Las entradas de campo deben ser únicas, así que se deben hacer un filtrado. 

In [10]:
print('Se tienen', df['Identificador'].count(), 'entradas.')

Se tienen 30000 entradas.


In [11]:
print('Solo', df['Identificador'].nunique(), 'don únicas')

Solo 18101 don únicas


In [12]:
# Se Eliminan datos repetidos
df = df.drop_duplicates(subset=['Identificador'],  keep= False)

In [13]:
#Se elije la comuna Identificador como Index 
df = df.set_index('Identificador')

In [14]:
#Se eliminan entradas con valores Nan
df = df.dropna()

In [15]:
df.head()

,periodo,DIAS1,DIAS2,DIAS3,DIAS4,DIAS5,DIAS6,DIAS0,Numero_Producto,GENERO,...,Saldo1,Saldo_Mora3,Saldo2,Saldo_Mora4,Saldo3,Saldo_Mora5,Saldo4,Saldo_Mora6,Saldo5,Saldo6
Identificador,,,,,,,,,,,,,,,,,,,,,
92966498191,2017-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,F,...,826282.0,22918.0,880038.0,21547.0,875198.0,22184.0,859556.0,23081.0,844034.0,827736.0
92966442488,2017-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,M,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
92967417881,2018-06,42.0,73.0,101.0,134.0,164.0,192.0,10.0,2,M,...,10262.0,800.0,10534.0,117.0,10249.0,10607.0,20572.0,23824.0,21573.0,81389.0
92966457920,2018-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,F,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,42020.0
92966454154,2017-12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,M,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
#Información de Data Set
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 781 entries, 92966498191 to 92966442639
Data columns (total 23 columns):
 #   Column           Non-Null Count  Dtype    
---  ------           --------------  -----    
 0   periodo          781 non-null    period[M]
 1   DIAS1            781 non-null    float64  
 2   DIAS2            781 non-null    float64  
 3   DIAS3            781 non-null    float64  
 4   DIAS4            781 non-null    float64  
 5   DIAS5            781 non-null    float64  
 6   DIAS6            781 non-null    float64  
 7   DIAS0            781 non-null    float64  
 8   Numero_Producto  781 non-null    int64    
 9   GENERO           781 non-null    object   
 10  Saldo_Mora1      781 non-null    float64  
 11  Cupo             781 non-null    float64  
 12  Saldo_Mora2      781 non-null    float64  
 13  Saldo1           781 non-null    float64  
 14  Saldo_Mora3      781 non-null    float64  
 15  Saldo2           781 non-null    float64  
 16  Saldo_Mo

In [17]:
print('La base de datos tiene:', len(df), 'entradas.')

La base de datos tiene: 781 entradas.


# 2. Análisis Descriptivo

Se hace una breve descrioción de las variables númericas del problema

In [18]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
DIAS1,781.0,4.472471e+00,1.802337e+01,0.0,0.0,0.0,0.0,118.0
DIAS2,781.0,5.692702e+00,2.324083e+01,0.0,0.0,0.0,0.0,134.0
DIAS3,781.0,6.955186e+00,2.831457e+01,0.0,0.0,0.0,0.0,163.0
DIAS4,781.0,8.562100e+00,3.387629e+01,0.0,0.0,0.0,0.0,195.0
DIAS5,781.0,1.012420e+01,4.033731e+01,0.0,0.0,0.0,0.0,226.0
DIAS6,781.0,1.134187e+01,4.488852e+01,0.0,0.0,0.0,0.0,254.0
DIAS0,781.0,4.329065e+00,1.384161e+01,0.0,0.0,0.0,0.0,86.0
Numero_Producto,781.0,1.204866e+00,5.896778e-01,1.0,1.0,1.0,1.0,6.0
Saldo_Mora1,781.0,7.817730e+03,7.576060e+04,0.0,0.0,0.0,0.0,2028684.0
Cupo,781.0,1.542386e+06,1.458452e+07,0.0,0.0,0.0,0.0,313138080.0


# 3. Feature Engineering

In [19]:
df_features = pd.DataFrame([])

## Saldo promedio de los ultimos 6 meses

In [20]:
df_features['Saldo_Promedio'] = df[['Saldo1','Saldo2','Saldo3','Saldo4','Saldo5','Saldo6']].mean(axis=1)

## Saldo máximo en el primer trimestre.

In [21]:
df_features['Saldo_Max'] = df[['Saldo1','Saldo2','Saldo3']].max(axis=1)

## Flag género

In [22]:
df_features['Flag_Genero'] = df.GENERO.eq('F').mul(1)

## Porcentaje promedio de saldo en mora con respecto al cupo en los 6 meses.

In [23]:
s_m_prom = df[['Saldo_Mora1','Saldo_Mora2','Saldo_Mora3','Saldo_Mora4','Saldo_Mora5','Saldo_Mora6']].mean(axis=1)
s_m_max = df[['Saldo_Mora1','Saldo_Mora2','Saldo_Mora3','Saldo_Mora4','Saldo_Mora5','Saldo_Mora6']].max(axis=1)
df_features['Porc_prom_Saldo_mora'] = s_m_prom/s_m_max 

In [24]:
#Cuando s_m_max es 0 significa que no tiene saldo de mora, por lo tanto el porcentaje promedio de saldo 
#en mora es 0
df_features['Porc_prom_Saldo_mora'] = df_features['Porc_prom_Saldo_mora'].fillna(0)

# 4. Segmentation Class
Se crean dos etiquetas para los clientes tomando 0 como clientes buenos y 1 como clientes malos  

In [25]:
#Clientes que que luego de los 6 meses hayan alcanzado un máximo de 30 días en mora
df_features['Tipo_Cliente_1'] = 0

In [26]:
for i, cli in enumerate(df_features['Tipo_Cliente_1']):
    if df['DIAS6'].iloc[i] > 30: df_features['Tipo_Cliente_1'].iloc[i]= 1

df_features['Tipo_Cliente_1'].value_counts()

/home/juanfe/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


0    731
1     50
Name: Tipo_Cliente_1, dtype: int64

In [27]:
#Clientes que que luego de los 6 meses hayan alcanzado un máximo de 90 días en mora.
df_features['Tipo_Cliente_2'] = 0

In [28]:
for i, cli in enumerate(df_features['Tipo_Cliente_2']):
    if df['DIAS6'].iloc[i] > 90: 
        df_features['Tipo_Cliente_2'].iloc[i]= 1

df_features['Tipo_Cliente_2'].value_counts()

0    740
1     41
Name: Tipo_Cliente_2, dtype: int64

# 5. Realice un modelo de regresión logística con las variables construidas en el punto 3 contra una de las variables respuesta del punto anterior.

In [30]:
#Variable categorica
y_1 = df_features['Tipo_Cliente_1'].values
y_2 = df_features['Tipo_Cliente_2'].values

In [31]:
#Caracterisiticas del problema 
X = df_features.iloc[:,:4].values

In [32]:
#Entranamiento del problema
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y_1, test_size=0.33, random_state=42)

In [35]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)

# Evaluación del modelo

In [ ]:
Çfrom sklearn.metrics import classification_report
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

# 6. Responda las siguientes preguntas:
• Justifique la elección de la variable respuesta, según su criterio.

Elegí el primer tipo de cliente para tener una primera impresión del comportamiento del problema con un flitro de tipo de cliente no tan exigente, para luego subir el límite.

Las variables del Género la elegí porque queria saber si la capidad de endedamiento tiene una correlación directa con está variable.

El saldo promedio me pareció relevante debido a que, es la varaible que me dice que si tengo o no capidad para pagar.

El saldo en mora proimedio lo elegí debido a que si se tiene saldos en mora pequeños se espera que ese comportamiento siga así ser un buen cliente.



• Calcule la tasa de malos con la variable respuesta elegida (Malos/Total Registros).







In [ ]:
np.array(np.unique(y_pred, return_counts=True)).T

In [ ]:
print('Tasa de malos:', 16/(246+16))


• ¿Qué variables resultaron predictivas en el modelo?

La variable que se puede predicir con mayor facilidad es si un cliente va a ser bueno, ya que se tiene un problema con desbalance de clases, lo que quiere decir que se tiene una muestra pequeña de malos clientes para entrenar el modelo, y este comportamiento tambien es debido a que se puede mejorar las caracteristicas con las cuales se entrena el modelo.